<h1 style="background-color:black; color:white; padding:10px;">
    Notebook 02 - Dataset do histórico das vendas
</h1>

<hr style="background-color:transparent;height:4px;border:none;border-top:2px solid #c0c0c0;border-bottom:2px solid #c0c0c0;">

### Definições iniciais

<hr style="height:0px;border:none;border-top:1px solid #c0c0c0;">

- Habilitação de recarga de módulo editado.

In [1]:
%load_ext autoreload
%autoreload 2

<hr style="height:0px;border:none;border-top:1px solid #c0c0c0;">

- Pacotes de uso geral.

In [2]:
import sys
import dtale    # instalado com: conda install dtale -c conda-forge
import numpy as np
import pandas as pd
from itertools import product
from tqdm.notebook import tqdm
from pprint import pformat # , pprint, PrettyPrinter

<hr style="height:0px;border:none;border-top:1px solid #c0c0c0;">

- Funções do pacote `bricklink_api`.
- Pacote `bricklinp_api`: https://github.com/BrickBytes/bricklink_api .
- Referência da API do Bricklink: https://www.bricklink.com/v3/api.page .

In [3]:
from bricklink_api.auth import DefaultOAuth
from bricklink_api.catalog_item import get_price_guide, GuideType, NewOrUsed, Type

<hr style="height:0px;border:none;border-top:1px solid #c0c0c0;">

- Módulos em diretórios fora de `sys.path` .

In [4]:
# diretório com módulos utilitários
sys.path.append('d:/py_utils')

In [5]:
from pd_utils import (
    d_pd, exemplo_linha, resumo_tipos, resumo_categ, resumo_serie,
    DisplayPandas, ExemploLinha, ResumoTipos, ResumoCateg, ResumoSerie
)

<hr style="height:0px;border:none;border-top:1px solid #c0c0c0;">

- Módulos no diretório do notebook.

In [6]:
from tcc import (
    ofuscar_params, frame2pickle, pickle2frame, object2pickle, pickle2object, coletar_dict)

<hr style="height:0px;border:none;border-top:1px solid #c0c0c0;">

- Utilidades de exibição e resumo de objetos `pandas`.

In [7]:
# primeiro item, 4 items intermediárias, item final e dimensões de objeto pandas
h1s4t1 = DisplayPandas(head=1, sample=4, tail=1)

<hr style="height:0px;border:none;border-top:1px solid #c0c0c0;">

- Outras definições.

In [8]:
d = display
pdod = pd.options.display
pdoc = pd.option_context

<hr style="background-color:transparent;height:4px;border:none;border-top:2px solid #c0c0c0;border-bottom:2px solid #c0c0c0;">

### Módulo `tcc`: variáveis comuns a mais de um notebook

In [9]:
# módulo `tcc.py` na pasta do notebok com variáveis comuns aos notebooks
import tcc

In [10]:
# caminhos
d(
      tcc.p_dir_tcc    # diretório inicial TCC
    , tcc.p_dir_bricklink    # diretório dados Bricklink
    , tcc.p_arq_autent    # parâmetros autenticação API Bricklink
    , tcc.pckl_df_guia_precos    # arquivo pickle dataframe
)

WindowsPath('d:/jup_ws/tcc')

WindowsPath('d:/jup_ws/tcc/bricklink')

WindowsPath('d:/jup_ws/tcc/bricklink/params_autent.json')

WindowsPath('d:/jup_ws/tcc/bricklink/df_guia_precos.pickle')

In [11]:
# conteúdo ofuscado dos parâmetros de autenticação
ofuscar_params(tcc.params_autent)

{'consumer_key': '********************************',
 'consumer_secret': '********************************',
 'token_secret': '********************************',
 'token_value': '********************************'}

In [12]:
# string com códigos de peças de interesse que comporão o dataset
tcc.txt_nos_bricks

'14716 22886 2356 2357 2453 2453a 2453b 2454 2456 2465 3001 3001f1 3001old 3001oldb 3001oldf1 3001special 3002 3002f1 3002old 3003 3003f1 3003old 3004 3004f1 3004f2 3005 3005f1 3005f2 3005f3 3006 3007 30072 3008 3008f1 3008f2 3009 3009f1 3009f2 3009f3 3010 3010f1 3010f2 3010f3 30144 30145 30400 3065 3066 3067 3245b 3245c 3622 3622f1 3754 3755 4201 4202 4204 46212 49311 6111 6112 6212 6213 700 700e 700eD 700eD2 700eX 702 702old 733 733eX 772 bhol01 bhol02 bhol03 bhol04 bhol05 bhol06 crssprt01 crssprt02 crssprt03 crssprt04 x1214'

<hr style="background-color:transparent;height:4px;border:none;border-top:2px solid #c0c0c0;border-bottom:2px solid #c0c0c0;">

### Autenticação para API do Bricklink

- Referência API: https://www.bricklink.com/v3/api.page?page=auth

In [13]:
default_oauth = DefaultOAuth()
default_oauth.set(**tcc.params_autent)

<hr style="background-color:transparent;height:4px;border:none;border-top:2px solid #c0c0c0;border-bottom:2px solid #c0c0c0;">

### Históricos de vendas das peças de interesse

- Referência API: https://www.bricklink.com/v3/api.page?page=get-price-guide
- Para obter o histórico de vendas dos últimos 6 meses, parâmetro `guide_type` tem que ser `"sold"` (`GuideType.SOLD` na biblioteca `bricklink_api`).
- Para diferenciar vendas de peças usadas de vendas de peças novas, é preciso usar o parâmetro `new_or_used`, valores possíveis `"N"` e `"U"` (`NewOrUsed.NEW` e `NewOrUsed.USED` na  biblioteca `bricklink_api`). Não utilizar esse parâmetro na obtenção do histórico de vendas equivale a pesquisar vendas de peças novas.
- Os detalhes das vendas realizadas não informam a cor da peça vendida. Para diferenciar vendas por cores, é preciso usar o parâmetro `color_id` com todas cores da tabela de domínio de cores de peças.

<hr style="height:0px;border:none;border-top:1px solid #c0c0c0;">

- Exemplo de resultado da função `get_price_guide` .

In [14]:
print(
    '\n'.join(pformat(
        get_price_guide(
            type_=Type.PART,
            no='3005',    # brick 1 x 1
            color_id=1,    # white
            guide_type=GuideType.SOLD,
            new_or_used=NewOrUsed.USED,
            currency_code='USD')
        , sort_dicts=False).split('\n')[:16]),
    '... ] }'
)

{'meta': {'description': 'OK', 'message': 'OK', 'code': 200},
 'data': {'item': {'no': '3005', 'type': 'PART'},
          'new_or_used': 'U',
          'currency_code': 'USD',
          'min_price': '0.0003',
          'max_price': '1.0000',
          'avg_price': '0.0461',
          'qty_avg_price': '0.0429',
          'unit_quantity': 7155,
          'total_quantity': 171839,
          'price_detail': [{'quantity': 3,
                            'unit_price': '0.0300',
                            'seller_country_code': 'US',
                            'buyer_country_code': 'IT',
                            'date_ordered': '2021-05-01T10:54:24.000Z',
                            'qunatity': 3}, ... ] }


<hr style="height:0px;border:none;border-top:1px solid #c0c0c0;">

- Definição e exemplo de resultado da função `frame_price_guide_part_sold_usd`.

In [15]:
def frame_price_guide_part_sold_usd(no, color_id, new_or_used):
    '''Dataframe de histórico de vendas de peças com valores em USD.'''
    resp = get_price_guide(
            type_=Type.PART, guide_type=GuideType.SOLD, currency_code='USD', 
            no=no, color_id=color_id, new_or_used=new_or_used)
    if resp and resp['data']['price_detail']:
        df = (
            pd.DataFrame.from_records(resp['data']['price_detail'])
            .drop(columns='qunatity')
        )
        return df

In [16]:
d_pd(
    frame_price_guide_part_sold_usd(
        no='3005',    # brick 1 x 1
        color_id=1,    # white
        new_or_used=NewOrUsed.USED)
    , sample=2
)

,quantity,unit_price,seller_country_code,buyer_country_code,date_ordered
2324,4,0.0400,US,US,2021-06-26T17:37:58.787Z
4869,2,0.0250,US,US,2021-09-02T08:19:30.060Z


(7155, 5)

<hr style="height:0px;border:none;border-top:1px solid #c0c0c0;">

- Lista dos códigos das peças de interesse.
- Quantidade de peças de interesse.
- Peças de interesse são da categoria *brick*: https://www.bricklink.com/catalogList.asp?catType=P&catString=5

In [17]:
nos_bricks = tcc.txt_nos_bricks.split(' ') 
print(nos_bricks)
len(nos_bricks)

['14716', '22886', '2356', '2357', '2453', '2453a', '2453b', '2454', '2456', '2465', '3001', '3001f1', '3001old', '3001oldb', '3001oldf1', '3001special', '3002', '3002f1', '3002old', '3003', '3003f1', '3003old', '3004', '3004f1', '3004f2', '3005', '3005f1', '3005f2', '3005f3', '3006', '3007', '30072', '3008', '3008f1', '3008f2', '3009', '3009f1', '3009f2', '3009f3', '3010', '3010f1', '3010f2', '3010f3', '30144', '30145', '30400', '3065', '3066', '3067', '3245b', '3245c', '3622', '3622f1', '3754', '3755', '4201', '4202', '4204', '46212', '49311', '6111', '6112', '6212', '6213', '700', '700e', '700eD', '700eD2', '700eX', '702', '702old', '733', '733eX', '772', 'bhol01', 'bhol02', 'bhol03', 'bhol04', 'bhol05', 'bhol06', 'crssprt01', 'crssprt02', 'crssprt03', 'crssprt04', 'x1214']


85

<hr style="height:0px;border:none;border-top:1px solid #c0c0c0;">

- Coleta dos históricos de vendas das peças de interesse, em um dicionário, com todas combinações de modelo da peça (`no`), cor da peça (`color_id`) e estado da peça (`new_or_used`), permitindo retomada das consultas somente para combinações de parâmetros ainda não coletadas.

In [18]:
# carga da lista de cores, para montar todas combinações de `(no, color_id, new_or_used)`
df_lista_cores = pickle2frame(p_pickle=tcc.pckl_df_lista_cores)

Lido: d:\jup_ws\tcc\bricklink\df_lista_cores.pickle
Tamanho: 8494 bytes
Modifificado em: 27/10/2021 11:13:00


In [19]:
# todas combinações (produto cartesiano) de `(no, color_id, new_or_used)` para pesquisa
params_guia_precos = list(product(nos_bricks, df_lista_cores.index, NewOrUsed))

In [20]:
# dicionário `dct_guia_precos` armazena resultados de todas consultas realizadas com 
# `frame_price_guide_part_sold_usd` para todos parâmetros em `params_guia_precos`
dct_guia_precos = {}

In [21]:
# função `coletar_dict` só consulta parâmetros ainda não coletados e exibe barra de progresso.
# coletar_dict(
#     dct_coleta=dct_guia_precos, 
#     iter_tqdm=params_guia_precos, 
#     desc_tqdm='Importando históricos de vendas', 
#     unit_tqdm='histórico',
#     fn_postfix=lambda tpl: dict(no=tpl[0], color_id=tpl[1], new_or_used=tpl[2].value), 
#     fn_chave=lambda tpl: (tpl[0], tpl[1], tpl[2].value), 
#     fn_coleta=lambda tpl: frame_price_guide_part_sold_usd(
#               **dict(no=tpl[0], color_id=tpl[1], new_or_used=tpl[2]) )
# )

<img style="float:left;" src="data:image/png;base64, 
iVBORw0KGgoAAAANSUhEUgAAAnUAAAA0CAIAAABjDlnjAAAVtklEQVR4nO2dP2jszHbAz4S8VO/r
tbAuFNykcLNFWC3cZsEhTSBXemuwFAIfTnc7427lLXalzri7XcxNwpMWvGgdSBNyYZsLqyWBbVy8
xkSFBav+e1VeMSmkkUbSSPvX1/58z68ys5qZMzNnzpk5M5IJpRQQBEEQBDkof/baAiAIgiDIOwT9
K4IgCIIcnj9/bQEQAf/5v//1b3/4/S9/+uW1BUG+Bz/95qd//Kt/+Nu//JvXFgRBkEOC+9e3CDrX
H4pf/vTLv/3h968tBYIgBwb961sEneuPBo44grw/0L8iCIIgyOEp+tdorJEzN3oVWTZnYRNi+xs8
6I8IGZUe3Dg797zmhhuXjyB7E401klDS1YVdnKShq5Ue80dEG2dPpQXyifFj9gIAIveMVSicMutr
jNyzuKg1NaYPc3X5tqix/qhaJHGuNCWlfqavqZdvzt65hCVwDy/SNLG1qVOJ3HBE7pmgz7c1Vv6I
iH1B6GoVEq4tcIOe+U4UZkemaeWmiSYXwEZ+5BX2r2/chZe0MHJvl86zpzdfTSTkh2NhN/TWnFJK
6Xxodvj5srCJYuYeDl3tyJjmC/BHpDPIPfNJB+eZ0mcH9E+cBYmCR1Vugj9qGCdJhdagk7POG9UY
uWcNY7JJjRmWT+m1AgD+qGMO50lj2eyLxlrn0VkJRUraWM6l9GnKyumB6l4ogpprSgBY2J2BNaeU
+papCAzobrnyfZWUQJ+dpaK5IUDoasrSeaaU0pULxpFgUVWpEiIF2BLfLiwLQvfm0Vnd69JexfIk
mvYWKMwOTtNKnV/Zt75dmBQifsj4cLtPab9m1uWR9PtK56pcJwYCQQ4Jp6LKqQWTIACAePGnLK2h
mj4YjTVyZLSGFpfZtwnpPFpWL0uKvnnTntZtAjT1q+HU+8aMczjzQOs2eU1WukOYPgWweY2hq5GG
cWLxQlTWKMCfDcA6VQBA+dlRB7PYugVPUziRpZJI9bk4OT8Z4Hw+FzgItpESl+B/NWHYVQCgfeH0
zFlxy7VbLo5w5k2sedzbTd2jnt7k/gCQPmgqLIPCiqRCJUQKIGY7Y9XUvSrnyom6BUzTXhvB7JDO
Pcoay3d+Td9G45tlTy2nF6j2rwubENtlEQl7UYp+lB7gG5BLDF2N2PaIEELsf7Yb+hQmRiNef4Vu
GvJgu8bIPSPayGbp2VIiE+ArJ2dWQlXIImCBL/ZAtq8vxsSisdYZAAw6iTDlwvm2LHKbXUEsSyRb
9tgb3sQjb4fMcAPA6ZxS7+KY+1m+WlHaP81l6fqU3l/IXArnqwAg81XRNw8+dvNmNPMfG9coXz1T
et3lk6pqFBAGS+B3Nol1k49VeAyivEjRWGNzU5wrbcWdPrUuix4inn2zU9pvV5UQBY+gHstcagTA
Rci3yiUi+uZNe7Jc9TO/NKkgpxIiBQAAgFkhiJ0ZK87qsqOBjglgKonFzgLRqY1a2OTMts8IIZr7
P3x8WBDoFmTnNW1dGDwhdDWi2SNWFBdTLFlazoxzwvgjzn1wZ3zl2VFJVd+G7qcH7fNla20B9ftX
04PPlNKVq5oKmZ3GoQkwb9NeM42nK5qERGLpI/ess3RX+UQAMJfHK0pp/5/6K1eFXhx5iNxLo+XT
OE6iDm7S7p4O4CqJ8LAITBp+oSv5Md2Y+/ZRUoI4qAIAAw9uY8mnxmXOpUXjTywmtnIeO/YCpHNv
PgQYzum1Ul04a0ubLyqNMKycxziWJcq+sNNAxPzE+CQ4l0IQxsImhHQGqvNzYkuVdnH/IbWV0iZD
UdrFJIDM+svH2bo7eJq2MicYm6SO2XMuWAkb1dhUFJEzENZYQSvxWE05NVrSuUcvgwYhhNzIz8l0
k849bhMmyBUTjW/4VgDnWfMzV1wC6xNJPmFJ7T7N9nMb56riRA6Eh7WhqxHSEK0MEkoqIVIAAICp
HnQLJjQhZ3WXiu2DpN/PLQDLp/02H4heOWA00rwTEy4ppZ7e4IpKrX0a6K7IzjTNt1kYnPqtgkEu
N8KM/cCzow46qe8vnRpI+j0t0G+Dcp0G8H1bMS0/3nOLZ0eK/8VIFzdVfet/MVpVA5Sn3r9aV+cS
AEhyC8DqtgGKU4UNc/vC6U29bxGEM2+iah+kXCIAAEvMIen3TNfDgI9xs1OTTFO5JZukX7IN+2Jm
MsGk8ysLRGGZ4VUcyhBP8sSpc5KkVBZebks0e5iqyT5A0u+pdy5VZp8YdwsAAOWaeqLgFYIktPuJ
GTra5TrJBvizQaKiMcp1svJrvIHgij8i5FZeUUrpZ7gUXpKqgp+PEO9vbkpr4tdk0Ik9fX4TEode
Y19VcRVoY5Uom1CeZI/U1L3SSVnR0maB95yqAEDO2rOgcUV2XtOmxhc/acua813mX7J1DD+yyoWr
Th9mNWqhXM+tQYeQjjmcbzL00VjrDFTntlaqhd2BjUqDvc9f2TouHcUwmGaJ68l2+rfL2iVuPvzS
lNUkdQm1kZZ6pHNv5YJxJA5WbFN4EEyA3wdUZm/3qW+ZimjpiiBCcuvU3UkjtMETW8qGwVKk4crP
jjrxZnt7dEGNlbDobhgskxR/NgC2jZP0S2uq35WiU+VcSYXBhF8ES/o9vXpqiGacuAQW3Y2Cxw2l
3SQXR7q3Fg5uU78agvm1+o7UXioh6fcrB4yG+JSqGOiuQ2DtK7Jnmqb0aezzilHfzRBY2qr4MADE
PhiAnXbXEo21hg7rrrJG7u0yDR6sZU//mp55MJVqyoKT+SpC92agJrGCW632UUk+4Y5w2GZXklvp
Of9uSOcepeLo8TaFy3KveOJSmb2d3HBcueq6e4YIAhU2ZTsKwZvYAooOXwFg61Xy5jWKKUZ3N6u6
JtdiZpYKibfm3a+c8RWXUNzwFXt+t1wcUjGYLXAY65zcniqRBlTn1qRwSpW3tPUIrL04e17T2B1v
Luq7MQJLWxUfBkgusavFCLmAaKw19NZ87b2tcOZNpsmWTDEBzE7tKfKe/pWtoRZ3RrxgbHa1Ximx
toR4ePwva+6XK6cWi+WC/5Wdv7a7adw1Gt+Y5QhGLfztJPlY5a9jbFm41P2YRSqSYkXZ+XeTJLm1
z+Ybec9wr9btoNhlpA9acr8hdG8G8ayMZg+QTk9uLkTuLXd95pA1VqF0h0nM0P9iTJOqlW521UMo
kjAXQBw6qpA/9bL2orIE5ZTtlRd3xqTc87vl4mh3LXZIlD3Mv1QqtJwHUwn+VRxZ7hX9tHKaBnXX
aQJv7VmxouycpvHNbMot2PaNHd7S+nc6fwpQJnIvDXA/e/dzq96RL+yGDs7zBi+VxAF8msT2Aax6
l7zn9/2t1lODEAAAy6d6EwAk/X4ekAbRucS8e5c+aKpuNEgwp/2rodFRiAlg+XNr0AlCgCpZ2/2V
qzWOiAGgDi22Nlb6z452FIugbtRBHMr1yjlL5Aew5lSBWL2UDnl0Vvf65oVL5978iTSIAQDQc1b3
CgAIsvOPgeo89/EAFhEQa3uimlsrtoCm/tn1kunjrvQmAATBpNW9T37PzYWes9r/rTNBjZUo13OL
dMgA0mmYipRMluTKIURjrfF0Ff8tzAVJOPqqtrrkaXEJ7f58SDrEBADLZ6kLm9zK8fugu+Xi6+/T
uU1Sy+ApANDUPT8gR4lpYOaUa+/BVIK3mQDDOW1DvJrpKGTprrxzrqI1mpCz9qq78toAUM7uZ5qW
b6bqrvpNAPBtciNv9o2BnKUdzmnlFZbIPWsY4KzOJQCp71tE0aqqiDdsBpMK0qtemyMeaCB05/+v
vrCJAvMtXiRFNuXv/v3ja4uAfG/+4+8fXluE70nknjWCyzdz4eiliNzRrHt9wK80vE+isT370P81
f8NHPNA/5PclEARBvgPhLDiuCWAiMdHsSX4Dn57Yg4qBRv+KIMjrYCrv/fPdTb2P7+CtR9J/7Vv8
ioHe4/y13d85tIwgyI+NpN9T/bWFQJAXBfevb5GffvPTa4uAfFdwxBHk/bHn/WHkRfjdkfov//2v
f/y/P762IMj34Ld/8dvf/fX6b4UjCPLrYo/7wwiCIAiCVIDxYQRBEAQ5POhfEQRBEOTwoH9FEARB
kMOD/hVBEARBDg/6VwRBEAQ5POhfEQRBEOTwoH9FEARBkMOD/hVBEARBDg/6VwRBEAQ5POhfEQRB
EOTwoH9FEARBkMNT51+jsUbO3Oi7yfIylfojoo258kJXI4QQkq/Ft+NEYqf/jjIaxw9qbsilvO9/
V4n8yojcs7x6b0voapyGb16sPyIMLvvCFiRmk2s/Ud8BC5u3MGX8UfEf4pZT1rJDlo3KJMReQDTW
dhnEspotbCJQvC1FOlAza51O5J4RQnas653vX/0R6Qz4hMi9NMBdUbpywPjEFMUfdczhnFI6H5qd
uB9D95PemlNK/ZZxGXe9f6e35tfK924DgrwxorHWeXRWlFJKVy4YR7YPAKGrKUvnOZ8IkXvWWbor
Sil9dkBv2ItXFv7H4WAeKHRvHp0VXcm3pKGD9mH/f4UeubdL59nTmweQTlzBwfZpkn5PV+6O/97q
HftX3yak82hZPS4tnHkTVfsgAUj6pTV9mMWOczYA61QBAOVnRx3MEpXsyTKXNRrfLN0L9K7Iu6Kp
e3RrMyede/Rej62s9EFTYRmEuaKyRJD0e+qdSwAAza7Wg2XwY29hXx7lmtLDbgOaunevSyDp95Ru
ry1JCbmMkn7/gs717VD2r2w7TLS7Jy41CZYSwaIgF/TgIktpJDaNA4SuRjR7xJKztZW4UlEMKn0y
y14Vtu36lN5f8D4SwmAKLTkd10kQAEAYLEHNEhO7wH4FgBNZAv9Ob12d779wQ5DNKE8ffkaIFL78
qz8i2sjWhNM2q4WfnvEcvwtYkWloVxT4rST65k17WrdgQLPVbQULmxDbZq3g4pCZGPYCYiPA9sG+
Tfi/a8UTW6SkxqqNNb8L5HZFAkMEFQOUJtpfISunwK4bzaxGNsSxwNFY6wwABh0y8vPhWb7HuFYI
jsY4tcm6jhe1MC5QqTC8ANkhQsV5wf6av7Ab+hQmRiOWXyz89p5uB2ielatCL478zC2A5G/fArDm
lFJK50OA4TyfaW4BWD6NQ0AqqM5zPjHN/uyowLI/Oyp7oLLSJDGrdOWqrPaV02Pl17FyeqDG4al8
mfTZUTOpLNakuZXIn8YEVOeZrlw1KwRBXhzB9ClMEzU+5mDqLfqVzocAmW6LyCYsN1N8CwA2VPj5
ELI5Rdkch8LcXDlxGKloOvLw9foWlAVj0mZ2wLfUXvZ3TpKSpBZvcDiLVN9S3uKlnSw0RNwQFDoz
HoK4E+qGo2xdRfaWI28ks0FPTrt4Y5uYtWI/cK0oCZyWwBnbSoVJyxeR8wvsSa4ivsWbaX7eKZS6
vZBYEn6tp1s5vfy416tuBYX9azR7mKofuxIAgHLBgs7+V1NlodFcBDVB6Q7B/BofWwbT4ZXeBFjM
zJ5z0QYAgPaF0zNnyQJHdX5WAACacqu2Umj3WQwqCh652gY3Los79dvbLSa2Qjr3KKWUenoz2byy
VVLdDQUEOQCLmQlWN5k+fUr7SmmasNONmOpfh92NYoX85rJ94fTWPR/XOtY6A9W51bM9aVP3KKXP
zlLhd4RxaHHlPHbWbdSsJErUlNVMsDRRvxpOvW+RJLfgMYgAomDZuryyHoMoNlNJD4jgulQ6v7Ig
s0g7HigWDVE0e5hal3FXZMdP/leTDYGkX1q7VFTPxLhbAAAo1ywOvzHSucciyUEwSRI5gdNQM2ds
w2AaP1dUmKn3bZP9ntJngeIoWAp+30nzy93Ot1IgfLWnyw1W0dNtTcG/BsEEWnIySFLiAaPgEaZ6
I9k1HxnTNILKSJ2u/9WMDzKjYAkTg+VpGJOacxdhpfym/pPHHpXOvZULxlExerAFadQ362vIYsKh
YMyTk9fkkJ+u3GUHbxEjL0kULAvH/4VpUqL+1w3IHZ1I8kn8R118OBprDR3Et1Sa+lW65s7YyWaF
wRTMDqu+M4DpUwDtrjXxZmE0ewC5KcvgzcIoeKzzlKIu3R2RIQqCCZgKE1QxYRIEEAWPoB6zatmi
oSo+LB+r8bphU9p96ltppVvfHctCtTfLZEWVF5iRBWBvl2zdkz9rYw+uO1DIItKfHgQS7ab5pW5f
J3y1pyu3fR8K/lWWuQsIbH0hySeFEEppRjW7Ws+cLfzZgK0Q5VYhVlO9thJWCv4XY5psyb2rk+xp
tq2k8+GU3ezdmHRRHBMPZLaTjikoDTt5Zf44XTsjyAshya2SmchNkxL1v25AU1azdXMaMVL6xYgX
23yMtYbemlfedhGbql2cXFNWC1HEawVA6Q6nQRgEoHWbUvcjBN9mHpROfDlEXboXJUMky8UTq74C
knwC0ydWbWpDWN58o8psYPHbyRCtXNVUtgqtRe6tyQz7Z401KydwIrZ7M2BB3Vv2YE5hUioVJmFx
Z0yS0fQuW1BiN80vdfs64as9XbHt+1Hwr1L3ozrV7+KL9bOHZIOnnFosMV55lUdR6n5UTaWT3bBt
dy0WuIh3otVrK3GlAJC4sdC9Ye/Y8HcN5GMVTrZcsTe7Ws+8GUeJbrH4QHc4Nb74kDj1XDwtuzac
rj2D5db1IshWtLtZADO5GyJ1P6aRMf9On+ZjofW/bkCzq/WSWQDhzJvUPrywGzo4z0VDxt1huTOS
4CH/Ku3OgsVzFuLtUVyafKyatzfLE1kCkOTW8sGD+pK5Lo3GN1kQch3yscr23Jl1EhkiqftRNW+5
S0ZnbgSgnFoskgz+V7O+LumDpk6y9wa5bhTDv4Uiya3KtUtTbgGL3y5mvBCJO1ncGWzEldMsxsCV
P41dqf/FSLqg2dXSmPA6OfPEXjlyb0W9sZPml7s9T0n4ak+XjrV7a256tlJDcbWR3IkAALCG2eE5
9wJQxTl27gg9TWF58tcT4nos7hKEoNI0e8+Z5w+rGck6qPbwOX+/KV8slzq38mVy6VkKE7L2wgiC
HARu+pSnieBaiuDXdbdjqGBKxhVaw7pbP1lFBQl9q5iSK5YTRngXib/wIhaMK4G/mlQ2PpWNzRux
jTKmtauOa9UYonzPZImp8VSHVnEvXick5I0tdzOIF25YfDgb9Hg40gtZSQc62X2fdLyG8+I1rnwr
Mvvs83Y7G5c1V03zt6ISaf2Kzt9I83O6Lej2pBBrXin8Ok/H6efO95sIpRQQBEG+K5E7mnWvdYwD
bU40tmcf+t/ntVF/pAU/H+4V1dDVjjztJT8o8aJEY63xdLXDW8Xv+PsSCIK8VcJZcLxloPhHJ5o9
yTUHzIdFuf4Ml3t9v7BE+TLU+wf9K4L8ACxK9zp3voF/EJp6/2W+1sJ9lIZj/bcC+C8tFK/1vg0k
/Xtt96OxRkjDqL0vtl1pR8V7Lb8eIveMNPTp+gdFYHwYQRAEQQ4P7l8RBEEQ5PCgf0UQBEGQw4P+
FUEQBEEOz/8DcV3vWZAKUPwAAAAASUVORK5CYII=
" />

<hr style="height:0px;border:none;border-top:1px solid #c0c0c0;">

- Carga de `dct_guia_precos` a partir de arquivo pickle, ao invés da coleta online.

In [23]:
# depois da importação inicial (demorada), dicionário foi salvo em arquivo pickle
# e pode ser carregado desse arquivo a qualquer tempo sem necessidade de nova coleta.
dct_guia_precos = pickle2object(tcc.pckl_dct_guia_precos)

Lido: d:\jup_ws\tcc\bricklink\dct_guia_precos.pickle
Tamanho: 73119862 bytes
Modifificado em: 30/07/2021 20:12:57


<hr style="height:0px;border:none;border-top:1px solid #c0c0c0;">

- Dataframe `df_guia_precos` com histórico de vendas das peças de interesse.
- Troca:
    - de `seq`: número de sequencial de um item de pedido dentro do grupo `(no, color_id, new_or_used)`
    - por `ord_itm_id`: número de sequencial de todos itens de pedidos reordenados por `date_ordered` (mantendo ordem da coleta original dentro da mesma `date_ordered`).

In [24]:
df_guia_precos = (
    pd.concat(dct_guia_precos, names=['no','color_id','new_or_used', 'seq'])
    .sort_values(by='date_ordered')
    .pipe(lambda df: df
        .assign(ord_itm_id=np.arange(df.shape[0]))
        .reset_index(level='seq', drop=True)
        .set_index('ord_itm_id', append=True)
    )
)

<hr style="height:0px;border:none;border-top:1px solid #c0c0c0;">

- Amostra e dimensões do dataframe.

In [25]:
d_pd(df_guia_precos, head=3, sample=3, tail=3)

,,,,quantity,unit_price,seller_country_code,buyer_country_code,date_ordered
no,color_id,new_or_used,ord_itm_id,,,,,
3005,80,U,0,10,0.0250,PL,PL,2016-12-13T18:42:52.050Z
3010,11,U,1,2,0.0550,SE,CA,2016-12-14T18:03:46.683Z
3065,12,U,2,1,0.0700,SE,CA,2016-12-14T18:03:46.683Z
3003,156,N,800205,1,0.0700,US,US,2021-05-04T08:21:18.383Z
3001,2,U,992748,4,0.1200,UK,UK,2021-06-01T13:26:22.820Z
3005,150,N,1134553,3,0.0910,AU,AU,2021-06-26T01:28:31.253Z
3622,85,U,1332486,200,0.0690,CZ,IE,2021-07-29T00:36:49.513Z
6111,86,U,1332487,2,0.7500,US,US,2021-07-29T00:41:34.067Z
30145,11,N,1332488,1,0.4051,AU,AU,2021-07-29T01:37:47.683Z


(1332489, 5)

<hr style="height:0px;border:none;border-top:1px solid #c0c0c0;">

- Resumo técnico.

In [26]:
df_guia_precos.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1332489 entries, ('3005', 80, 'U', 0) to ('30145', 11, 'N', 1332488)
Data columns (total 5 columns):
 #   Column               Non-Null Count    Dtype 
---  ------               --------------    ----- 
 0   quantity             1332489 non-null  int64 
 1   unit_price           1332489 non-null  object
 2   seller_country_code  1332489 non-null  object
 3   buyer_country_code   1332489 non-null  object
 4   date_ordered         1332489 non-null  object
dtypes: int64(1), object(4)
memory usage: 102.1+ MB


<hr style="height:0px;border:none;border-top:1px solid #c0c0c0;">

- Salvamento e teste da carga do dataframe em formato pickle.

In [27]:
frame2pickle(frame=df_guia_precos, p_pickle=tcc.pckl_df_guia_precos)

Salvo: d:\jup_ws\tcc\bricklink\df_guia_precos.pickle
Tamanho: 91964293 bytes
Modifificado em: 27/10/2021 13:31:45


In [28]:
_ = pickle2frame(p_pickle=tcc.pckl_df_guia_precos)
del _

Lido: d:\jup_ws\tcc\bricklink\df_guia_precos.pickle
Tamanho: 91964293 bytes
Modifificado em: 27/10/2021 13:31:45
